# Accessing/Customizing Prompts within Higher-Level Modules

LlamaIndex contains a variety of higher-level modules (query engines, response synthesizers, retrievers, etc.), many of which make LLM calls + use prompt templates.

This guide shows how you can 1) access the set of prompts for any module (including nested) with `get_prompts`, and 2) update these prompts easily with `update_prompts`.

In [ ]:
import os
import openai

In [ ]:
os.environ["OPENAI_API_KEY"] = "sk-..."
openai.api_key = os.environ["OPENAI_API_KEY"]

In [ ]:
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

from llama_index import (
    VectorStoreIndex,
    SimpleDirectoryReader,
    load_index_from_storage,
    StorageContext,
)
from IPython.display import Markdown, display

INFO:numexpr.utils:Note: NumExpr detected 12 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
Note: NumExpr detected 12 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
INFO:numexpr.utils:NumExpr defaulting to 8 threads.
NumExpr defaulting to 8 threads.


## Setup: Load Data, Build Index, and Get Query Engine

Here we build a vector index over a toy dataset (PG's essay), and access the query engine.

The query engine is a simple RAG pipeline consisting of top-k retrieval + LLM synthesis.

In [ ]:
# load documents
documents = SimpleDirectoryReader(
    "../../../examples/paul_graham_essay/data"
).load_data()

In [ ]:
index = VectorStoreIndex.from_documents(documents)

In [ ]:
# set Logging to DEBUG for more detailed outputs
query_engine = index.as_query_engine(response_mode="tree_summarize")

## Accessing Prompts

Here we get the prompts from the query engine. Note that *all* prompts are returned, including ones used in sub-modules in the query engine. This allows you to centralize a view of these prompts!

In [ ]:
prompts_dict = query_engine.get_prompts()

In [ ]:
display({k: p.get_template() for k, p in prompts_dict.items()})

{'response_synthesizer:summary_template': 'Context information from multiple sources is below.\n---------------------\n{context_str}\n---------------------\nGiven the information from multiple sources and not prior knowledge, answer the query.\nQuery: {query_str}\nAnswer: '}

#### Checking `get_prompts` on Response Synthesizer

You can also call `get_prompts` on the underlying response synthesizer, where you'll see the same list.

In [ ]:
prompts_dict = query_engine.response_synthesizer.get_prompts()
display({k: p.get_template() for k, p in prompts_dict.items()})

{'summary_template': 'Context information from multiple sources is below.\n---------------------\n{context_str}\n---------------------\nGiven the information from multiple sources and not prior knowledge, answer the query.\nQuery: {query_str}\nAnswer: '}

#### Checking `get_prompts` with a different response synthesis strategy

Here we try the default `compact` method.

We'll see that the set of templates used are different; a QA template and a refine template.

In [ ]:
# set Logging to DEBUG for more detailed outputs
query_engine = index.as_query_engine(response_mode="compact")

In [ ]:
prompts_dict = query_engine.get_prompts()
display({k: p.get_template() for k, p in prompts_dict.items()})

{'response_synthesizer:text_qa_template': 'Context information is below.\n---------------------\n{context_str}\n---------------------\nGiven the context information and not prior knowledge, answer the query.\nQuery: {query_str}\nAnswer: ',
 'response_synthesizer:refine_template': "The original query is as follows: {query_str}\nWe have provided an existing answer: {existing_answer}\nWe have the opportunity to refine the existing answer (only if needed) with some more context below.\n------------\n{context_msg}\n------------\nGiven the new context, refine the original answer to better answer the query. If the context isn't useful, return the original answer.\nRefined Answer: "}

#### Put into query engine, get response

In [ ]:
response = query_engine.query("What did the author do growing up?")
print(str(response))

The author worked on writing and programming outside of school before college. They wrote short stories and tried writing programs on an IBM 1401 computer using an early version of Fortran. They later got a microcomputer and started programming on it, writing simple games and a word processor. They also mentioned their interest in philosophy and AI.


## Customize the prompt

You can also update/customize the prompts with the `update_prompts` function. Pass in arg values with the keys equal to the keys you see in the prompt dictionary.

Here we'll change the summary prompt to use Shakespeare.

In [ ]:
from llama_index.prompts import PromptTemplate

# reset
query_engine = index.as_query_engine(response_mode="tree_summarize")

# shakespeare!
new_summary_tmpl_str = (
    "Context information is below.\n"
    "---------------------\n"
    "{context_str}\n"
    "---------------------\n"
    "Given the context information and not prior knowledge, "
    "answer the query in the style of a Shakespeare play.\n"
    "Query: {query_str}\n"
    "Answer: "
)
new_summary_tmpl = PromptTemplate(new_summary_tmpl_str)

In [ ]:
query_engine.update_prompts(
    {"response_synthesizer:summary_template": new_summary_tmpl}
)

In [ ]:
prompts_dict = query_engine.get_prompts()

In [ ]:
display({k: p.get_template() for k, p in prompts_dict.items()})

{'response_synthesizer:text_qa_template': 'Context information is below.\n---------------------\n{context_str}\n---------------------\nGiven the context information and not prior knowledge, answer the query.\nQuery: {query_str}\nAnswer: ',
 'response_synthesizer:refine_template': "The original query is as follows: {query_str}\nWe have provided an existing answer: {existing_answer}\nWe have the opportunity to refine the existing answer (only if needed) with some more context below.\n------------\n{context_msg}\n------------\nGiven the new context, refine the original answer to better answer the query. If the context isn't useful, return the original answer.\nRefined Answer: "}

In [ ]:
response = query_engine.query("What did the author do growing up?")
print(str(response))

## Accessing Prompts from Other Modules

Here we take a look at some other modules: query engines, routers/selectors, evaluators, and others.

In [ ]:
from llama_index.query_engine import (
    RouterQueryEngine,
    FLAREInstructQueryEngine,
)
from llama_index.selectors import LLMMultiSelector
from llama_index.evaluation import FaithfulnessEvaluator, DatasetGenerator
from llama_index.indices.postprocessor import LLMRerank

#### Analyze Prompts: Router Query Engine

In [ ]:
# setup sample router query engine
from llama_index.tools.query_engine import QueryEngineTool

query_tool = QueryEngineTool.from_defaults(
    query_engine=query_engine, description="test description"
)

router_query_engine = RouterQueryEngine.from_defaults([query_tool])

In [ ]:
prompts_dict = router_query_engine.get_prompts()
display({k: p.get_template() for k, p in prompts_dict.items()})

{'summarizer:summary_template': 'Context information from multiple sources is below.\n---------------------\n{context_str}\n---------------------\nGiven the information from multiple sources and not prior knowledge, answer the query.\nQuery: {query_str}\nAnswer: '}

#### Analyze Prompts: FLARE Query Engine

In [ ]:
flare_query_engine = FLAREInstructQueryEngine(query_engine)

In [ ]:
prompts_dict = flare_query_engine.get_prompts()
display({k: p.get_template() for k, p in prompts_dict.items()})

{'instruct_prompt': 'Skill 1. Use the Search API to look up relevant information by writing     "[Search(query)]" where "query" is the search query you want to look up.     For example:\n\nQuery: But what are the risks during production of nanomaterials?\nAnswer: [Search(What are some nanomaterial production risks?)]\n\nQuery: The colors on the flag of Ghana have the following meanings.\nAnswer: Red is for [Search(What is the meaning of Ghana\'s flag being red?)],     green for forests, and gold for mineral wealth.\n\nQuery: What did the author do during his time in college?\nAnswer: The author took classes in [Search(What classes did the author take in     college?)].\n\n\n\nSkill 2. Solve more complex generation tasks by thinking step by step. For example:\n\nQuery: Give a summary of the author\'s life and career.\nAnswer: The author was born in 1990. Growing up, he [Search(What did the     author do during his childhood?)].\n\nQuery: Can you write a summary of the Great Gatsby.\nAns

#### Analyze Prompts: LLMMultiSelector

In [ ]:
from llama_index.selectors.llm_selectors import LLMSingleSelector

selector = LLMSingleSelector.from_defaults()

In [ ]:
prompts_dict = selector.get_prompts()
display({k: p.get_template() for k, p in prompts_dict.items()})

{'prompt': "Some choices are given below. It is provided in a numbered list (1 to {num_choices}), where each item in the list corresponds to a summary.\n---------------------\n{context_list}\n---------------------\nUsing only the choices above and not prior knowledge, return the choice that is most relevant to the question: '{query_str}'\n"}

#### Analyze Prompts: FaithfulnessEvaluator

In [ ]:
evaluator = FaithfulnessEvaluator()

In [ ]:
prompts_dict = evaluator.get_prompts()
display({k: p.get_template() for k, p in prompts_dict.items()})

{'eval_template': "Please tell if a given piece of information is supported by the context.\nYou need to answer with either YES or NO.\nAnswer YES if any of the context supports the information, even if most of the context is unrelated. Some examples are provided below. \n\nInformation: Apple pie is generally double-crusted.\nContext: An apple pie is a fruit pie in which the principal filling ingredient is apples. \nApple pie is often served with whipped cream, ice cream ('apple pie à la mode'), custard or cheddar cheese.\nIt is generally double-crusted, with pastry both above and below the filling; the upper crust may be solid or latticed (woven of crosswise strips).\nAnswer: YES\nInformation: Apple pies tastes bad.\nContext: An apple pie is a fruit pie in which the principal filling ingredient is apples. \nApple pie is often served with whipped cream, ice cream ('apple pie à la mode'), custard or cheddar cheese.\nIt is generally double-crusted, with pastry both above and below the fi

#### Analyze Prompts: DatasetGenerator

In [ ]:
dataset_generator = DatasetGenerator.from_documents(documents)

In [ ]:
prompts_dict = dataset_generator.get_prompts()
display({k: p.get_template() for k, p in prompts_dict.items()})

{'text_question_template': 'Context information is below.\n---------------------\n{context_str}\n---------------------\nGiven the context information and not prior knowledge.\ngenerate only questions based on the below query.\n{query_str}\n',
 'text_qa_template': 'Context information is below.\n---------------------\n{context_str}\n---------------------\nGiven the context information and not prior knowledge, answer the query.\nQuery: {query_str}\nAnswer: '}

#### Analyze Prompts: LLMRerank

In [ ]:
llm_rerank = LLMRerank()

In [ ]:
prompts_dict = dataset_generator.get_prompts()
display({k: p.get_template() for k, p in prompts_dict.items()})

{'text_question_template': 'Context information is below.\n---------------------\n{context_str}\n---------------------\nGiven the context information and not prior knowledge.\ngenerate only questions based on the below query.\n{query_str}\n',
 'text_qa_template': 'Context information is below.\n---------------------\n{context_str}\n---------------------\nGiven the context information and not prior knowledge, answer the query.\nQuery: {query_str}\nAnswer: '}